In [1]:
import pandas as pd
import geopandas as gpd

c:\anaconda3\envs\geospatial\lib\site-packages\pyproj\__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [2]:
# Panda settings for showing data (this is foremost done to more easily explore the data while processing it)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
input_folder = "C://bomen_data//input//"
output_folder = "C://bomen_data//output//"

Translating the inspection data into English

In [ ]:
tree_inspections = pd.read_csv(input_folder+'bomen-veiligheidsinspecties-2024-11.csv')

In [ ]:
tree_inspections['Advice'] = tree_inspections['Snoeiwijze'].replace({
    'Begeleidingssnoei': 'Juvenile pruning',
    'Onderhoudssnoei': 'Maintenance pruning',
    'Knotten': 'Pollarding' ,
    'Fruitsnoei': 'Fruit pruning',
    'Terugzetten (extensief)': 'basic pruning',
    'Niet van toepassing': 'Not applicable',
    'Terugzetten (intensief)': 'Heavy pruning',
    'Scheren | knippen': 'Heavy pruning',
    'Kandelaberen': 'Topping',
    'Innemen kroon' : 'Reduding crown'
})

In [ ]:
tree_inspections.rename(columns={'Boomid': 'TreeId'}, inplace=True)
tree_inspections.rename(columns={'Inspecterendeorganisatie': 'Inspecting_company'}, inplace=True)
tree_inspections.rename(columns={'Inspectiedatum': 'Inspection_date'}, inplace=True)

In [ ]:
selected_columns = ['Id', 'TreeId','Inspecting_company','Inspection_date','Advice']
tree_inspections.to_csv(output_folder+'tree_inspections_11_2024.csv', columns=selected_columns, index=False)

Translating the tree november 2024 data into English

In [ ]:
tree_data_memory_nov24 = gpd.read_file(input_folder+'bomen-stamgegevens_2024_11.json')


In [22]:
tree_data_memory_oct24 = gpd.read_file(input_folder+'Trees_amsterdam_oct_2024.gpkg')
tree_data_memory_oct23 = gpd.read_file(input_folder+'Trees_Amsterdam_sept_2023.shp')

In [24]:
tree_data_memory_oct23.head()

,id,soortnaam,soortnaamK,SoortnaamN,boomhoogte,stamdiamet,jaarVanAan,typeObject,geometry
0,919934,Ulmus hollandica `Vegeta`,Ulmus,Huntingdon-iep,b. 6 tot 9 m.,None,1978,Boom niet vrij uitgroeiend,POINT (121980.449 483684.998)
1,919935,Fraxinus excelsior `Westhof`s Glorie`,Fraxinus,Gewone es (cultuurvariëteit),c. 9 tot 12 m.,"0,2 tot 0,3 m.",1990,Boom niet vrij uitgroeiend,POINT (118737.842 482805.464)
2,919936,Ulmus glabra `Lutescens`,Ulmus,Gele bergiep (cultuurvariëteit),b. 6 tot 9 m.,None,2002,Boom niet vrij uitgroeiend,POINT (122053.589 484657.432)
3,919937,Quercus robur,Quercus,Zomereik,b. 6 tot 9 m.,None,1985,Boom niet vrij uitgroeiend,POINT (120154.401 483804.692)
4,919939,Styphnolobium japonicum,Overig,Honingboom,a. tot 6 m.,None,2002,Boom niet vrij uitgroeiend,POINT (120453.549 482894.090)


In [13]:
tree_data = tree_data_memory

In [14]:
tree_data.head(5)

,id,gbdBuurtId,typeBeheerderPlus,boomhoogteklasseActueel,Owner,year_planted,botanical_name,stamdiameterklasse,standplaats,standplaatsGedetailleerd,typeObject,typeSoortnaam,soortnaamKort,name,geometry,diameter_tree_trunk,height_of_tree,object_type
0,919933,03630980000301,R&E_VOR_Bomen,e. 15 tot 18 m.,Gemeente Amsterdam,1948.0,Tilia americana,None,None,None,Boom niet vrij uitgroeiend,Bomen,Tilia,Linde (Tilia),POINT (4.90467 52.33981),None,e. 15 to 18,non free growing tree
1,919934,03630980000301,R&E_VOR_Bomen,c. 9 tot 12 m.,Gemeente Amsterdam,1978.0,Ulmus hollandica 'Vegeta',None,Elementenverharding,Tegels,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus),POINT (4.90269 52.34009),None,c. 9 to 12 m.,non free growing tree
2,919935,03630980000311,R&E_VOR_Bomen,c. 9 tot 12 m.,Gemeente Amsterdam,1990.0,Fraxinus excelsior 'Westhof's Glorie',"0,2 tot 0,3 m.",None,None,Boom niet vrij uitgroeiend,Bomen,Fraxinus,Es (Fraxinus),POINT (4.85521 52.33198),0.2 to 0.3 m.,c. 9 to 12 m.,non free growing tree
3,919936,03630980000297,R&E_VOR_Bomen,b. 6 tot 9 m.,Gemeente Amsterdam,2002.0,Ulmus glabra 'Lutescens',None,Heesters,Fijne sierheester,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus),POINT (4.90368 52.34883),None,b. 6 to 9 m.,non free growing tree
4,919937,03630980000306,R&E_VOR_Bomen,b. 6 tot 9 m.,Gemeente Amsterdam,1985.0,Quercus robur,None,Bosplantsoen,Struikvormers,Boom niet vrij uitgroeiend,Bomen,Quercus,Eik (Quercus),POINT (4.87589 52.34106),None,b. 6 to 9 m.,non free growing tree


In [6]:
diameter_trunks = tree_data['stamdiameterklasse'].unique()
diameter_trunks

array([None, '0,2 tot 0,3 m.', '0,1 tot 0,2 m.', '0,3 tot 0,5 m.',
       '0,5 tot 1 m.', '1,0 tot 1,5 m.', 'Onbekend', '1,5 m. en groter',
       'tot 0,1 m.'], dtype=object)

In [15]:
tree_data['diameter_tree_trunk'] = tree_data['stamdiameterklasse'].replace({
     '0,2 tot 0,3 m.': "0.2 to 0.3 m.",
     '0,1 tot 0,2 m.': "0.1 to 0.2 m.",
     '0,3 tot 0,5 m.': "0.3 to 0.5 m.",
     '0,5 tot 1 m.': "0.5 to 1 m.",
     '1,0 tot 1,5 m.':"1.0 to 1.5 m.",
     'Onbekend': "unknown",
     '1,5 m. en groter': "1.5 m. and bigger",
     'tot 0,1 m.': "smaller than 0.1 m."
})

In [ ]:
tree_heigth = tree_data['boomhoogteklasseActueel'].unique()
tree_heigth

In [16]:
tree_data['height_of_tree'] = tree_data['boomhoogteklasseActueel'].replace({
     'e. 15 tot 18 m.': 'e. 15 to 18 ',
     'c. 9 tot 12 m.': 'c. 9 to 12 m.' ,
     'b. 6 tot 9 m.': 'b. 6 to 9 m.' ,
     'd. 12 tot 15 m.': 'd. 12 to 15 m.',
     'a. tot 6 m.': 'a. to 6 m.' ,
     'f. 18 tot 24 m.': 'f. 18 to 24 m.',
     'q. Niet van toepassing': 'q. Not applicable',
     'g. 24 m. en hoger': 'g. 24 m. and higher',
     'p. Niet te beoordelen': 'p. Impossible to determine'  ,
     'r. Niet van toepassing (te vellen boom)': 'r. Not applicabel (tree need to be removed)' ,
     'ZZ_Vervallen 6 tot 12 m.': 'ZZ_Discard 6 to 12 m.' ,
     'ZZ_Vervallen 12 tot 18 m.': 'ZZ_Discard 12 to 18 m.'
})

In [ ]:
tree_heigth = tree_data['typeObject'].unique()
tree_heigth

In [ ]:
tree_data['object_type'] = tree_data['typeObject'].replace({
     'Boom niet vrij uitgroeiend': 'non free growing tree',
     'Boom vrij uitgroeiend':'free growing tree',
     'Stobbe':'Tree Stub',
     'Gekandelaberde boom': 'Topped tree',
     'Knotboom': 'Pollarded tree',
     'Vormboom': 'Topiary tree',
     'Fruitboom': 'Fruit tree',
     'Leiboom':'Espalier tree',
     'Bosplantsoen': 'Forest park',
     'Aanwijzingsteken' : 'Indication sign'

})

In [ ]:
tree_data_5 = tree_data.head(5)

In [ ]:
tree_data_5.to_file(output_folder+'output.geojson', driver='GeoJSON')

In [20]:
tree_data.head(5)

,id,Owner,year_planted,botanical_name,name,geometry,diameter_tree_trunk,height_of_tree,object_type
0,919933,Gemeente Amsterdam,1948.0,Tilia americana,Linde (Tilia),POINT (4.90467 52.33981),None,e. 15 to 18,non free growing tree
1,919934,Gemeente Amsterdam,1978.0,Ulmus hollandica 'Vegeta',Iep (Ulmus),POINT (4.90269 52.34009),None,c. 9 to 12 m.,non free growing tree
2,919935,Gemeente Amsterdam,1990.0,Fraxinus excelsior 'Westhof's Glorie',Es (Fraxinus),POINT (4.85521 52.33198),0.2 to 0.3 m.,c. 9 to 12 m.,non free growing tree
3,919936,Gemeente Amsterdam,2002.0,Ulmus glabra 'Lutescens',Iep (Ulmus),POINT (4.90368 52.34883),None,b. 6 to 9 m.,non free growing tree
4,919937,Gemeente Amsterdam,1985.0,Quercus robur,Eik (Quercus),POINT (4.87589 52.34106),None,b. 6 to 9 m.,non free growing tree


In [19]:
tree_data.rename(columns={'typeEigenaarPlus': 'Owner'}, inplace=True)
tree_data.rename(columns={'jaarVanAanleg': 'year_planted'}, inplace=True)
tree_data.rename(columns={'soortnaam': 'botanical_name'}, inplace=True)
tree_data.rename(columns={'soortnaamTop': 'name'}, inplace=True)
tree_data.rename(columns={'soortnaam': 'botanical_name'}, inplace=True)

tree_data = tree_data.drop(columns=['gbdBuurtId','typeObject','boomhoogteklasseActueel','stamdiameterklasse','typeBeheerderPlus','standplaats', 'standplaatsGedetailleerd', 'typeSoortnaam','soortnaamKort'])

In [ ]:
tree_soortnaam = tree_data['typeSoortnaam'].unique()
tree_soortnaam

In [21]:
tree_data.to_file(output_folder+'trees_nov_2024.geojson', driver='GeoJSON')

Translating the tree October 2024 data into English

In [23]:
tree_data_memory_oct24.head()

,Boomnummer,Soortnaam,SoortnaamKortNL,SoortnaamTop15,Jaar_van_aanleg,Boomhoogte,Type_detail,Standplaats_detail,Beheerder,geometry
0,1106188,Ulmus hollandica `Vegeta`,Hollandse iep,Iep,1981,18 tot 24 m.,Boom niet vrij uitgroeiend,Gras- en kruidachtigen - Gazon,Gemeente,POINT (4.82107 52.34210)
1,1106189,Fraxinus excelsior `Westhof`s Glorie`,Gewone es,Es,1981,18 tot 24 m.,Boom niet vrij uitgroeiend,Gras- en kruidachtigen - Gazon,Gemeente,POINT (4.82122 52.34211)
2,1106190,Ulmus hollandica `Vegeta`,Hollandse iep,Iep,1981,18 tot 24 m.,Boom niet vrij uitgroeiend,Gras- en kruidachtigen - Gazon,Gemeente,POINT (4.82127 52.34212)
3,1106191,Fraxinus excelsior cv.,Gewone es,Es,1981,12 tot 15 m.,Boom niet vrij uitgroeiend,Gras- en kruidachtigen - Ruw gras,Gemeente,POINT (4.82462 52.34227)
4,1106192,Ulmus hollandica `Vegeta`,Hollandse iep,Iep,1981,15 tot 18 m.,Boom niet vrij uitgroeiend,Gras- en kruidachtigen - Ruw gras,Gemeente,POINT (4.82454 52.34226)


Translating the tree October 2023 data into English

In [25]:
tree_data_memory_oct23.head()

,id,soortnaam,soortnaamK,SoortnaamN,boomhoogte,stamdiamet,jaarVanAan,typeObject,geometry
0,919934,Ulmus hollandica `Vegeta`,Ulmus,Huntingdon-iep,b. 6 tot 9 m.,None,1978,Boom niet vrij uitgroeiend,POINT (121980.449 483684.998)
1,919935,Fraxinus excelsior `Westhof`s Glorie`,Fraxinus,Gewone es (cultuurvariëteit),c. 9 tot 12 m.,"0,2 tot 0,3 m.",1990,Boom niet vrij uitgroeiend,POINT (118737.842 482805.464)
2,919936,Ulmus glabra `Lutescens`,Ulmus,Gele bergiep (cultuurvariëteit),b. 6 tot 9 m.,None,2002,Boom niet vrij uitgroeiend,POINT (122053.589 484657.432)
3,919937,Quercus robur,Quercus,Zomereik,b. 6 tot 9 m.,None,1985,Boom niet vrij uitgroeiend,POINT (120154.401 483804.692)
4,919939,Styphnolobium japonicum,Overig,Honingboom,a. tot 6 m.,None,2002,Boom niet vrij uitgroeiend,POINT (120453.549 482894.090)
